<a href="https://colab.research.google.com/github/ErickQuinteroOsorio/edp_con_pinn/blob/main/notebooks/analisis_edp_con_pinn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Listing 1: Initial settings for the heat equation
#----------------------------------------------------
import tensorflow as tf
# We set seeds initially. This feature controls the randomization of
# variables (e.g. initial weights of the network).
# By doing it so, we can reproduce same results.
tf.random.set_seed(123)
# 100 equidistant points in the domain are created
x = tf.linspace(0.0, 1.0, 100)
# boundary conditions T(0)=T(1)=0 and \kappa are introduced.
bcs_x = [0.0, 1.0]
bcs_T = [0.0, 0.0]

bcs_x_tensor = tf.convert_to_tensor(bcs_x)[:, None]
bcs_T_tensor = tf.convert_to_tensor(bcs_T)[:, None]
kappa = 0.5

# Number of iterations
N = 1000
# ADAM optimizer with learning rate of 0.005
optim = tf.keras.optimizers.Adam(learning_rate=0.005)
# Function for creating the model
def buildModel(num_hidden_layers, num_neurons_per_layer):
  tf.keras.backend.set_floatx("float32")

  # Initialize a feedforward neural network
  model = tf.keras.Sequential()
  # Input is one dimensional (one spatial dimension)
  # model.add(tf.keras.Input(1)) # así estaba en el libro y ya no funciona
  model.add(tf.keras.Input(shape=(1,))) # Corrected line

  # Append hidden layers
  for _ in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_neurons_per_layer,
                                    activation=tf.keras.activations.get("tanh"),
                                    kernel_initializer="glorot_normal",
                                    )
    )

  # Output is one-dimensional
  model.add(tf.keras.layers.Dense(1))
  return model


# determine the model size (3 hidden layers with 32 neurons each)
model = buildModel(3, 32)

In [16]:
# Listing 2: Loss function definition for the heat equation
#------------------------------------------------------------
# Boundary loss function
def boundary_loss(bcs_x_tensor, bcs_T_tensor):
  predicted_bcs = model(bcs_x_tensor)
  mse_bcs = tf.reduce_mean(tf.square(predicted_bcs - bcs_T_tensor))
  return mse_bcs

# the first derivative of the prediction
def get_first_deriv(x):
  with tf.GradientTape() as tape:
    tape.watch(x)
    T = model(x)
  T_x = tape.gradient(T, x)
  return T_x

# the second derivative of the prediction
def second_deriv(x):
  with tf.GradientTape() as tape:
    tape.watch(x)
    T_x = get_first_deriv(x)
  T_xx = tape.gradient(T_x, x)
  return T_xx

# Source term divided by \kappa
source_func = lambda x: (15 * x - 2) / kappa

# Function for physics loss
def physics_loss(x):
  predicted_Txx = second_deriv(x)
  mse_phys = tf.reduce_mean(tf.square(predicted_Txx + source_func(x)))
  return mse_phys

# Overall loss function
def loss_func(x, bcs_x_tensor, bcs_T_tensor):
  bcs_loss = boundary_loss(bcs_x_tensor, bcs_T_tensor)
  phys_loss = physics_loss(x)
  loss = bcs_loss + phys_loss
  return loss

In [24]:
# Listing 3: Training of the heat equation model
#-------------------------------------------------
# taking gradients of the loss function
def get_grad():
  with tf.GradientTape() as tape:
    # This tape is for derivatives with
    # respect to trainable variables
    # tape.watch(model.trainable_variables) # Removed redundant watch
    Loss = loss_func(x, bcs_x_tensor, bcs_T_tensor)

  g = tape.gradient(Loss, model.trainable_variables)
  return Loss, g

# optimizing and updating the weights of the model by using gradients
def train_step():
  # Compute current loss and gradient w.r.t. parameters
  loss, grad_theta = get_grad()
  # Perform gradient descent step
  optim.apply_gradients(zip(grad_theta, model.trainable_variables))
  return loss

# Training loop
for i in range(N + 1):
  loss = train_step()
  # printing loss amount in each 100 epoch
  if i % 100 == 0:
    print("Epoch {:05d}: loss = {:10.8e}".format(i, loss))

Epoch 00000: loss = 1.99108078e+02
Epoch 00100: loss = 1.53024683e+01
Epoch 00200: loss = 3.80971245e-02
Epoch 00300: loss = 3.48418951e-03
Epoch 00400: loss = 1.71695207e-03
Epoch 00500: loss = 1.28656975e-03
Epoch 00600: loss = 1.09738461e-03
Epoch 00700: loss = 2.69013480e-03
Epoch 00800: loss = 9.23037704e-04
Epoch 00900: loss = 1.22378662e-03
Epoch 01000: loss = 1.18407246e-03


Aora con la red ya entrenada

Once the training process is completed with the desired loss value, we can validate
the output by performing one forward pass with a test dataset which is typically
formed in the same domain as the training dataset. In our example, the training data
was 100 equidistant points between 0 and 1.We can determine our test dataset as 200
equidistant points in the same domain. Figure 5.5 depicts that the model’s prediction
captures the analytical result.